In [2]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, load_tool, tool
import datetime
import requests
import pytz
import yaml

c:\Users\mathe\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
import re
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
import numpy as np

In [3]:
# Configurações
class Config:
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2" # modelo de enbedding usado texto -> vetor numérico
    LLM_MODEL = "llama3.2:3b" # modelo
    TEMPERATURE = 0.5 # 0.5 menor = mais deterministico / maior = criativo ou aleatório
    CHUNK_SIZE = 300 # 1500 chunks para divisão do texto
    CHUNK_OVERLAP = 50 # 200 sobreposição dos pedaços de texto
    PERSIST_DIR = "./chroma_db_pt_legal" # diretorio dos enbeddings
    PDF_PATHS = ["M:\\tcc\\edge-ai\\pdfs\\classificacao.pdf"]
    # PDF_PATHS = ["M:\\tcc\\edge-ai\\pdfs\\Direção Defensiva.pdf", 
    #             "M:\\tcc\\edge-ai\\pdfs\\Manual de Primeiros Socorros.pdf",
    #             "M:\\tcc\\edge-ai\\pdfs\\Noções de Primeiros Socorros no Trânsito.pdf",
    #             "M:\\tcc\\edge-ai\\pdfs\\Segurança no Transporte crianças e gestantes.pdf",
    #             "M:\\tcc\\edge-ai\\pdfs\\Sinalização.pdf"]

In [4]:
# Função de limpeza (mesma do código anterior)
def clean_brazilian_legal_text(text: str) -> str:
    patterns = [
        r"Diário Oficial .+? Página \d+",
        r"Lei Nº \d+\.\d+ de \d{2}/\d{2}/\d{4}",
        r"Publicado em: \d{2}/\d{2}/\d{4}",
        r"Este texto não substitui o original publicado",
        r"\n\s*\d+\s*\n"
    ]
    
    for pattern in patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)
    
    text = re.sub(r"(?i)(artigo|art\.) ?(\d+)", r"Art. \2", text)
    text = re.sub(r"§ ?(único|\d+º?)", r"§ \1", text)
    
    return re.sub(r"\s+", " ", text).strip()

In [5]:
# Carregar e processar documentos
documents = []
for path in Config.PDF_PATHS:
    try:
        loader = PyPDFLoader(path)
        pages = loader.load_and_split(text_splitter=None)
        for page in pages:
            cleaned = clean_brazilian_legal_text(page.page_content)
            if cleaned.strip():
                page.page_content = cleaned
                documents.append(page)
        print(f"✓ {os.path.basename(path)} processado")
    except Exception as e:
        print(f"✗ Erro em {path}: {str(e)}")

✓ classificacao.pdf processado


In [6]:
# Configuração do text splitter jurídico corrigida
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=Config.CHUNK_SIZE,
    chunk_overlap=Config.CHUNK_OVERLAP,
    separators=[
        r"\n\nArt\. \d+\.",  # Separa por artigos
        r"\n§ ",             # Separa por parágrafos
        r"\n\n", 
        r"\n", 
        " ", 
        ""
    ],
    length_function=lambda x: len(x.split()),
    is_separator_regex=False  # Desativa regex complexo
)

In [7]:
# Processamento de documentos
texts = text_splitter.split_documents(documents)

# Verificação pós-split
print(f"Total de chunks gerados: {len(texts)}")
print("Exemplo de chunk inicial:")
print(texts[0].page_content[:500] + "...")

Total de chunks gerados: 126
Exemplo de chunk inicial:
1 Manual de Acolhimento e Classificação de Risco MANUAL DE ACOLHIMENTO E CLASSIFICAÇÃO DE RISCO Secretaria de Estado de Saúde do Distrito Federal 1a Versão...


In [8]:
# Verificar duplicatas
seen = set()
duplicates = 0
for t in texts:
    h = hash(t.page_content.strip().lower())
    if h in seen:
        duplicates += 1
    seen.add(h)
print(f"Chunks duplicados: {duplicates}")

Chunks duplicados: 0


In [9]:
# Inicializar embeddings
embeddings = HuggingFaceEmbeddings(model_name=Config.EMBEDDING_MODEL)

#### O que é ChromaDB?
##### 📌 ChromaDB é um banco de dados vetorial que permite armazenar e buscar textos embeddados.
##### 📌 Ele é otimizado para busca semântica, onde textos semelhantes retornam resultados próximos.
##### 📌 Armazena os embeddings persistentemente, evitando precisar recalcular sempre.

In [10]:
# Criar vetorstore
db = Chroma.from_documents(
    texts,
    embeddings,
    persist_directory=Config.PERSIST_DIR,
    collection_metadata={"hnsw:space": "cosine"}
)

In [11]:
# Configurar retriever
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 1,
        "lambda_mult": 0.45,
        "score_threshold": 0.25
    }
)

In [12]:
# Inicializar LLM
llm = OllamaLLM(
    model=Config.LLM_MODEL,
    temperature=Config.TEMPERATURE,
    system="Você é uma enfermeira especialista em triagens. Responda com base no documento fornecido."
    # system="Você é um especialista em legislação de trânsito brasileira. Responda com base nos documentos fornecidos."
)

In [13]:
# Prompt template
prompt_template = """
    Analise os seguintes trechos e responda apenas com a cor da sua prioridade na classificação:

    Contexto:
    {context}
    Pergunta: {question}
    
    Resposta:
"""

#Inclua:
#    1. Base legal (artigos/parágrafos)
#    2. Explicação técnica
#    3. Fontes (documento e página)
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [14]:
# Criar cadeia QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [15]:
def consultar(pergunta):
    try:
        result = qa_chain.invoke({"query": pergunta})
        
        print(f"\n🔍 Pergunta: {pergunta}")
        print(f"\n📝 Resposta:\n{result['result']}")
        
        print("\n📚 Fontes:")
        seen_sources = set()
        for doc in result['source_documents']:
            src = f"{os.path.basename(doc.metadata['source'])} - Página {doc.metadata['page']}"
            if src not in seen_sources:
                print(f"  - {src}")
                seen_sources.add(src)
                
    except Exception as e:
        print(f"Erro: {str(e)}")

In [26]:
consultar("Considere um paciente com dor moderada (entre 4 e 6 em uma escala de 0 a 10) que melhora com analgésicos, febre entre 37,5°C e 38°C sem sinais graves de toxemia, deficiência neurológica de evolução lenta há mais de 24 horas, presença de sangue na urina sem outros sinais graves, vômitos e diarreia moderados sem sinais de desidratação grave, pressão arterial elevada sem sintomas de AVC ou crise hipertensiva, ferimentos leves com sangramento controlado, dificuldade respiratória leve sem sinais de insuficiência respiratória grave e histórico de alergia leve sem sinais de anafilaxia. Com base nesses sintomas, qual seria a prioridade de atendimento desse paciente de acordo com o protocolo de classificação de risco?")


🔍 Pergunta: Considere um paciente com dor moderada (entre 4 e 6 em uma escala de 0 a 10) que melhora com analgésicos, febre entre 37,5°C e 38°C sem sinais graves de toxemia, deficiência neurológica de evolução lenta há mais de 24 horas, presença de sangue na urina sem outros sinais graves, vômitos e diarreia moderados sem sinais de desidratação grave, pressão arterial elevada sem sintomas de AVC ou crise hipertensiva, ferimentos leves com sangramento controlado, dificuldade respiratória leve sem sinais de insuficiência respiratória grave e histórico de alergia leve sem sinais de anafilaxia. Com base nesses sintomas, qual seria a prioridade de atendimento desse paciente de acordo com o protocolo de classificação de risco?

📝 Resposta:
Laranja.

📚 Fontes:
  - classificacao.pdf - Página 69


In [31]:
# Exemplo 2
consultar("Quais as penalidades por dirigir sob efeito de álcool?")


🔍 Pergunta: Quais as penalidades por dirigir sob efeito de álcool?

📝 Resposta:
Aqui estão as penalidades por dirigir sob efeito de álcool, conforme a legislação brasileira:

1. **Base Legal**: De acordo com o Código de Trânsito Rodoviário Brasileiro (CRTRB), artigo 305, § 2º, "O condutor que se encontra em estado de embriaguez, sob efeito de álcool ou qualquer outro medicamento, será punido com multa, desde que não haja morte ou lesão corporal." Além disso, o artigo 305, § 3º, estabelece que "A multa será aplicada ao condutor, independentemente da presença de danos à vítima."
2. **Explicação Técnica**: Dirigir sob efeito de álcool é considerado um crime grave, pois aumenta o risco de acidentes e lesões. O álcool pode reduzir a coordenação motora, a percepção sensorial e a capacidade de tomar decisões rápidas, tornando mais difícil controlar o veículo.
3. **Fontes**:
 * Documento: Código de Trânsito Rodoviário Brasileiro (CRTRB) - Lei nº 9.664, de 27 de maio de 1998
 * Página: [www.tr

In [21]:
# Exemplo 3
consultar("Compare as exigências para as categorias B e D")


🔍 Pergunta: Compare as exigências para as categorias B e D

📝 Resposta:
**Comparação das Exigências para Categorias B e D**

**Base Legal:**

*   Categoria B: Artigo 60, § 1º do CTB (Código de Trânsito Brasileiro) - "A via deve ser classificada como urbana ou rural com base nos seguintes critérios:
    *   Via urbana: via que recebe o tráfego de veículos motorizados e pedestres, sem a presença de tráfego de carga;
    *   Via rural: via que recebe o tráfego de veículos motorizados e pedestres, com a presença de tráfego de carga."
*   Categoria D: Artigo 60, § 2º do CTB - "A categoria D é reservada para vias rurais que recebem o tráfego de veículos motorizados e pedestres, com a presença de tráfego de carga."

**Explicação Técnica:**

*   Categoria B:
    *   É utilizada em via urbana;
    *   Aplique-se a velocidade máxima permitida é de 60 km/h.
*   Categoria D:
    *   É utilizada em via rural;
    *   Aplique-se a velocidade máxima permitida é de 80 km/h.

**Fontes:**

*   Document

In [22]:
consultar(
    '''
    Escolha a opção correta: Para habilitar-se na categoria “D”, o candidato deverá ser: 
    a) maior de 18 anos; 
    b) penalmente imputável, não importando sua idade; 
    c) menor de 21 anos; 
    d) maior de 21 anos"
    '''
)


🔍 Pergunta: 
    Escolha a opção correta: Para habilitar-se na categoria “D”, o candidato deverá ser: 
    a) maior de 18 anos; 
    b) penalmente imputável, não importando sua idade; 
    c) menor de 21 anos; 
    d) maior de 21 anos"
    

📝 Resposta:
**Base Legal:**
O artigo 10 da Lei nº 9.394, de 4 de setembro de 1997, que estabelece as diretrizes para os programas de educação básica, inclui a seguinte disposição: "A habilitação para dirigir veículos automotores é obrigatória para todos os candidatos que pretendem obter o certificado de habilitação para dirigir."

Além disso, o artigo 12 da Lei nº 9.394/97 estabelece que a idade mínima para obter a habilitação para dirigir é de 18 anos.

**Explicação Técnica:**
A habilitação para dirigir é um documento que autoriza uma pessoa a operar veículos automotores em estradas públicas. Para obter essa habilitação, o candidato deve atender a certos requisitos legais, incluindo idade.

O artigo 10 da Lei nº 9.394/97 estabelece que o candidat

In [23]:
consultar(
'''
    Escolha a opção correta: Para dirigir com segurança, evitando acidentes, o condutor deve demonstrar:
    
    Opção 1: habilidade ao dirigir; conhecimento das regras de trânsito; cooperação com os demais usuários da via;
    Opção 2: conhecimento de algumas regras de trânsito; agressividade nas situações perigosas; habilidade ao dirigir;
    Opção 3: bom senso; respeito apenas às regras mais importantes para a segurança; habilidade ao dirigir;
    Opção 4: habilidade ao dirigir; conhecimento de algumas regras de trânsito; bom senso.
    '''
)


🔍 Pergunta: 
    Escolha a opção correta: Para dirigir com segurança, evitando acidentes, o condutor deve demonstrar:
    
    Opção 1: habilidade ao dirigir; conhecimento das regras de trânsito; cooperação com os demais usuários da via;
    Opção 2: conhecimento de algumas regras de trânsito; agressividade nas situações perigosas; habilidade ao dirigir;
    Opção 3: bom senso; respeito apenas às regras mais importantes para a segurança; habilidade ao dirigir;
    Opção 4: habilidade ao dirigir; conhecimento de algumas regras de trânsito; bom senso.
    

📝 Resposta:
**Escolha a opção correta:**

Opção 4: habilidade ao dirigir; conhecimento de algumas regras de trânsito; bom senso.

**Razão:**

Para dirigir com segurança, é necessário demonstrar habilidade ao dirigir, conhecimento de algumas regras de trânsito e bom senso. Isso porque a habilidade ao dirigir é fundamental para controlar o veículo de forma eficaz, o conhecimento de algumas regras de trânsito ajuda a evitar acidentes e 

In [24]:
consultar(
    '''
    Escolha a opção correta: O condutor de veículo deve dar preferência de passagem aos pedestres:
    
    Opção 1: somente quando estão atravessando na faixa de pedestres;
    Opção 2: que não tenham concluído a travessia, quando houver mudança de sinal;
    Opção 3: caso as pessoas estejam próximas a área escolar;
    Opção 4: somente quando isso for solicitado pelo agente de trânsito.
    '''
)


🔍 Pergunta: 
    Escolha a opção correta: O condutor de veículo deve dar preferência de passagem aos pedestres:
    
    Opção 1: somente quando estão atravessando na faixa de pedestres;
    Opção 2: que não tenham concluído a travessia, quando houver mudança de sinal;
    Opção 3: caso as pessoas estejam próximas a área escolar;
    Opção 4: somente quando isso for solicitado pelo agente de trânsito.
    

📝 Resposta:
**Opção Correta:**

A opção correta é **Opção 3: caso as pessoas estejam próximas a área escolar.**

**Base Legal:**

O artigo 27, § 2º, do Código de Trânsito Brasileiro (CTB) estabelece que "o condutor de veículo deve dar preferência de passagem aos pedestres quando estes se encontrarem próximos a escolas ou à faixa de pedestres".

**Explicação Técnica:**

A direção defensiva é uma abordagem de condução que visa minimizar os riscos e garantir a segurança de todos os usuários da via. Dada a importância de proteger as crianças e adolescentes, especialmente em áreas escol

<img src="https://portal.wemeds.com.br/wp-content/uploads/2023/04/escore-news.jpg" width="500"/>

In [ ]:
@tool
def clinical_deterioration_risk(resp_rate: int, spo2: int, bp_systolic: int, heart_rate: int, temp: float, avpu: str) -> str:
    """Calcula o risco de deterioração clínica com base no escore NEWS.
    
    Args:
        resp_rate: Frequência respiratória (rpm)
        spo2: Saturação de oxigênio (%)
        bp_systolic: Pressão arterial sistólica (mmHg)
        heart_rate: Frequência cardíaca (bpm)
        temp: Temperatura corporal (°C)
        avpu: Nível de consciência (Alerta, Responde à Voz, Responde à Dor, Inconsciente)
    
    Returns:
        Uma string indicando o nível de risco do paciente.
    """

    if resp_rate <= 8:
        resp_score = 3
    elif 9 <= resp_rate <= 11:
        resp_score = 1
    elif 12 <= resp_rate <= 20:
        resp_score = 0
    elif 21 <= resp_rate <= 24:
        resp_score = 2
    else:
        resp_score = 3
    
    if spo2 >= 96:
        spo2_score = 0
    elif 94 <= spo2 <= 95:
        spo2_score = 1
    elif 92 <= spo2 <= 93:
        spo2_score = 2
    else:
        spo2_score = 3
    
    if bp_systolic >= 110 and bp_systolic <= 219:
        bp_score = 0
    elif 100 <= bp_systolic < 110:
        bp_score = 1
    elif 90 <= bp_systolic < 100:
        bp_score = 2
    else:
        bp_score = 3
    
    if heart_rate <= 40:
        hr_score = 3
    elif 41 <= heart_rate <= 50:
        hr_score = 1
    elif 51 <= heart_rate <= 90:
        hr_score = 0
    elif 91 <= heart_rate <= 110:
        hr_score = 1
    elif 111 <= heart_rate <= 130:
        hr_score = 2
    else:
        hr_score = 3
    
    if temp < 35:
        temp_score = 3
    elif 35 <= temp <= 36:
        temp_score = 1
    elif 36.1 <= temp <= 38:
        temp_score = 0
    elif 38.1 <= temp <= 39:
        temp_score = 1
    else:
        temp_score = 2
    
    avpu_score = 0 if avpu.lower() == "alerta" else 3
    total_score = resp_score + spo2_score + bp_score + hr_score + temp_score + avpu_score
    
    if total_score == 0:
        return "Risco mínimo - Monitoramento de rotina"
    elif 1 <= total_score <= 4:
        return "Risco baixo - Observação clínica"
    elif 5 <= total_score <= 6:
        return "Risco moderado - Revisão urgente por profissional de saúde"
    else:
        return "Risco alto - Atendimento médico imediato"